# Imports

In [1]:
import pandas as pd

In [2]:
import xml.etree.ElementTree as et

In [3]:
import os
import sys

In [4]:
import matplotlib.pyplot as plt

In [5]:
#import argparse

# Define and parse command-line arguments
#parser = argparse.ArgumentParser(description='Process some integers.')
#parser.add_argument('--seed', type=int, help='Seed value')
#parser.add_argument('--traffic_scale', type=float, help='Traffic scale value')
#args = parser.parse_args()

# Functions

In [6]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [7]:
price_per_wh = 110 / 1000

In [8]:
def get_price(d_energy):
    return d_energy * price_per_wh

In [9]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [10]:
def get_route_id(trip_id, trip_ids):
    trip_id = trip_id[:-2]
    line = trip_ids.loc[trip_ids['trip_id'] == trip_id]
    route_id = line.iloc[0]['route_id']

    return route_id

In [11]:
def get_routeids_df(base_folder):
    file_path = os.path.join(base_folder, "trips.txt")
    return pd.read_csv(file_path, sep=",")

In [12]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [13]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [14]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [15]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        route_dict['stops'] = stops
        
        yield route_dict

In [16]:
def get_group_by_id(list_of_dfs, desired_id):
    for df in list_of_dfs:
        if desired_id in df['id'].values:
            return df[df['id'] == desired_id]
    raise ValueError(f"ID '{desired_id}' not found in any dataframe.")

In [ ]:
def main(seed, scale):
    print(seed + " " + scale)

# XML to df

In [17]:
base_folder = "C:\\Users\\Admin\\Sumo\\nap_gellert_b"

In [18]:
file_path = os.path.join(base_folder, "emission.out.xml")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

#emission_output_df.shape

(45190, 20)

In [19]:
file_path = os.path.join(base_folder, "Battery.out.xml")

battery_output = et.parse(file_path)
battery_output_root = battery_output.getroot()

transform = transform_xml(battery_output_root)
battery_output_list = list(transform)

battery_output_pd = pd.DataFrame(battery_output_list)

battery_output_pd = battery_output_pd.drop(battery_output_pd.columns[0], axis=1)
#battery_output_pd

,id,energyConsumed,totalEnergyConsumed,totalEnergyRegenerated,actualBatteryCapacity,maximumBatteryCapacity,chargingStationId,energyCharged,energyChargedInTransit,energyChargedStopped,speed,acceleration,x,y,lane,posOnLane,timeStopped
0,C767841140.0,0.0000,0.0000,0.0000,1500.0000,3000.0000,NULL,0.0000,0.0000,0.0000,0.0000,0.0000,4054.5714,1655.1857,296910360#1_0,12.1000,0
1,C767841140.0,6.2090,6.2090,0.0000,1493.7910,3000.0000,NULL,0.0000,0.0000,0.0000,1.0000,1.0000,4053.6881,1654.7200,296910360#1_0,13.1000,0
2,C767841140.0,15.1682,21.3771,0.0000,1478.6229,3000.0000,NULL,0.0000,0.0000,0.0000,2.0000,1.0000,4051.9213,1653.7887,296910360#1_0,15.1000,0
3,C767841140.0,24.1416,45.5187,0.0000,1454.4813,3000.0000,NULL,0.0000,0.0000,0.0000,3.0000,1.0000,4049.2713,1652.3918,296910360#1_0,18.1000,0
4,C767841140.0,33.1278,78.6465,0.0000,1421.3535,3000.0000,NULL,0.0000,0.0000,0.0000,4.0000,1.0000,4045.7379,1650.5292,296910360#1_0,22.1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45185,C78105432.0,96.6069,9404.7804,2649.6308,672.6271,3000.0000,NULL,0.0000,0.0000,0.0000,8.0000,1.0000,3708.2978,3228.6050,25440988#0_0,49.0000,0
45186,C78105432.0,70.9618,9475.7422,2649.6308,601.6653,3000.0000,NULL,0.0000,0.0000,0.0000,8.3208,0.3208,3701.2316,3224.3130,25440988#0_0,57.3208,0
45187,C78105432.0,56.3178,9532.0599,2649.6308,545.3476,3000.0000,NULL,0.0000,0.0000,0.0000,8.3208,0.0000,3694.1654,3220.0211,25440988#0_0,65.6417,0
45188,C78105432.0,56.1577,9588.2177,2649.6308,489.1898,3000.0000,NULL,0.0000,0.0000,0.0000,8.3208,0.0000,3687.0992,3215.7291,25440988#0_0,73.9625,0


In [20]:
file_path = os.path.join(base_folder, "tripinfo.xml")

b_tripinfo_output = et.parse(file_path)
b_tripinfo_output_root = b_tripinfo_output.getroot()

transform = transform_xml_tripinfo(b_tripinfo_output_root)
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)

b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
#b_tripinfo_output_pd

,id,depart,departLane,departPos,departSpeed,departDelay,arrival,arrivalLane,arrivalPos,arrivalSpeed,...,routeLength,waitingTime,waitingCount,stopTime,timeLoss,rerouteNo,devices,vType,speedFactor,vaporized
0,C767841140.0,1970.00,296910360#1_0,12.10,0.00,0.00,2669.00,54727328_0,73.00,8.00,...,1012.78,556.00,4,20.00,607.70,0,tripinfo_C767841140.0 emissions_C767841140.0 b...,ElectricBus,1.06,
1,C76784379.0,10910.00,296910360#1_0,12.10,0.00,0.00,11519.00,54727328_0,73.00,8.00,...,1012.78,466.00,4,20.00,517.45,0,tripinfo_C76784379.0 emissions_C76784379.0 bat...,ElectricBus,1.05,
2,C6305325.0,12950.00,34572881#1_0,12.10,0.00,0.00,13520.00,-50219636#1_0,56.74,9.00,...,1702.65,346.00,4,30.00,409.49,0,tripinfo_C6305325.0 emissions_C6305325.0 batte...,ElectricBus,0.94,
3,C77632401.0,16070.00,34572881#1_0,12.10,0.00,0.00,16703.00,-50219636#1_0,56.74,9.00,...,1702.65,405.00,4,30.00,464.04,0,tripinfo_C77632401.0 emissions_C77632401.0 bat...,ElectricBus,0.87,
4,C7769412.0,18890.00,292524113#0_0,12.10,0.00,0.00,19352.00,600260151_0,113.30,10.00,...,1636.63,198.00,3,64.00,289.15,0,tripinfo_C7769412.0 emissions_C7769412.0 batte...,ElectricBus,1.09,
5,C78257148.0,23030.00,548875276#0_0,12.10,0.00,0.00,24197.00,-581325727_0,332.48,10.00,...,3311.51,618.00,7,95.00,751.02,0,tripinfo_C78257148.0 emissions_C78257148.0 bat...,ElectricBus,0.98,
6,C6359838.0,24410.00,34572881#1_0,12.10,0.00,0.00,24861.00,34572878#1_0,48.32,1.00,...,575.27,346.00,3,20.00,375.22,0,tripinfo_C6359838.0 emissions_C6359838.0 batte...,ElectricBus,1.09,
7,C76796152.0,25130.00,53864729#4_0,12.10,0.00,0.00,25195.00,53864729#4_0,445.46,1.00,...,433.36,1.00,1,20.00,16.97,0,tripinfo_C76796152.0 emissions_C76796152.0 bat...,ElectricBus,1.12,
8,C7810534.0,25610.00,581325727_0,12.10,0.00,0.00,25862.00,25440988#0_0,85.69,8.05,...,2169.76,0.00,0,20.00,20.95,0,tripinfo_C7810534.0 emissions_C7810534.0 batte...,ElectricBus,0.97,
9,C76647202.0,26330.00,894166783_0,4.14,0.00,0.00,27207.00,311385575_0,48.95,0.00,...,1055.47,706.00,7,20.00,771.96,0,tripinfo_C76647202.0 emissions_C76647202.0 bat...,ElectricBus,1.02,teleport


In [21]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [22]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

In [23]:
route_ids_df = get_routeids_df(base_folder)

## Grouping by id

In [24]:
grouped_df = battery_output_pd.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#list_of_dfs

[                 id energyConsumed totalEnergyConsumed totalEnergyRegenerated  \
 14518  C622152254.0         0.0000              0.0000                 0.0000   
 14520  C622152254.0         4.4999              4.4999                 0.0000   
 14522  C622152254.0        11.8490             16.3488                 0.0000   
 14524  C622152254.0        19.3771             35.7259                 0.0000   
 14526  C622152254.0        27.1621             62.8880                 0.0000   
 ...             ...            ...                 ...                    ...   
 15489  C622152254.0         0.0309           3827.6537              1677.7691   
 15491  C622152254.0         2.4277           3830.0814              1677.7691   
 15493  C622152254.0         6.8502           3836.9316              1677.7691   
 15495  C622152254.0        10.6362           3847.5678              1677.7691   
 15497  C622152254.0        15.1207           3862.6885              1677.7691   
 
       actualB

In [25]:
grouped_emission_df = emission_output_df.groupby('id')

list_of_emission_dfs = [group_data for _, group_data in grouped_emission_df]
#list_of_emission_dfs



[                 id                                eclass       CO2    CO  \
 14518  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C   2775.00  2.08   
 14520  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C   7696.00  3.90   
 14522  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C   9974.00  3.91   
 14524  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C  12252.00  3.93   
 14526  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C  14530.00  3.96   
 ...             ...                                   ...       ...   ...   
 15489  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C   2775.00  2.08   
 15491  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C   7696.00  3.90   
 15493  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C   9974.00  3.91   
 15495  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C  12252.00  3.93   
 15497  C622152254.0  HBEFA4/UBus_Std_gt15-18t_Euro-VI_A-C  14530.00  3.96   
 
          HC   NOx   PMx     fuel electricity  noise       rou

In [26]:
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1])
    #print(get_price(energy))

240.34113400000004
247.68594399999995
244.33481600000002
63.890057000000006
120.285319
722.38562
143.64564499999997
1289.378211
1261.057611
946.160149
426.16203300000006
942.2991710000001
982.9160880000001
768.015083
469.77328199999994
286.84539400000006
474.419671
368.59087100000005
224.85773200000003
185.94358200000002
181.030135
293.925258
204.44944300000003
213.73480700000002
232.701128
65.597488
65.29193
73.292659
68.94079500000001
-3.6596230000000047
-3.6596230000000047
75.438253
-3.6596230000000047
124.21688400000001
327.495872
3689.3388069999996
2313.2065
1399.782494
1259.401616
312.48765899999995
237.662975
192.800212
214.139387
713.123389
446.8029279999999
460.47443199999987
499.46377800000005
740.4771440000001
769.4219060000001
284.938324
521.936118
483.16807
289.570831
1620.8219060000001
1725.0778489999998
2338.624497
834.680594
835.101168
790.33977


In [27]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = get_price(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    #print(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    
    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    route_id = get_route_id(group_id, route_ids_df)
    
    z_up = calc_elevation_up(get_group_by_id(list_of_emission_dfs, group_id))
    z_down = calc_elevation_down(get_group_by_id(list_of_emission_dfs, group_id))
    
    # Store the results in a dictionary
    group_result = {
        'routeid': route_id,
        'id': group_id,
        'avgSpeed': avg_speed,
        'battery': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
#print(result_df)


2184.9194
2251.6903999999995
2221.2256
580.8187
1093.5029
6567.142
1305.8694999999998
11721.6201
11464.1601
8601.4559
3874.2003000000004
8566.3561
8935.6008
6981.9553
4270.6662
2607.6854000000003
4312.9061
3350.8261
2044.1612000000002
1690.3962000000001
1645.7285
2672.0478
1858.6313000000002
1943.0437000000002
2115.4648
596.3408
593.5629999999999
666.2968999999999
626.7345
-33.269300000000044
-33.269300000000044
685.8023000000001
-33.269300000000044
1129.2444
2977.2352
33539.443699999996
21029.149999999998
12725.2954
11449.1056
2840.7968999999994
2160.5724999999998
1752.7291999999998
1946.7217
6482.939899999999
4061.844799999999
4186.131199999999
4540.5798
6731.6104000000005
6994.744600000001
2590.3484000000003
4744.873799999999
4392.437
2632.4620999999997
14734.744600000002
15682.525899999999
21260.2227
7588.0054
7591.8288
7184.907


# CSV

In [28]:
tableBattery = pd.read_csv('batteryData.csv', delimiter=';')

#tableBattery.shape

(120, 12)

## settings

In [29]:
locSetting = get_last_folder(base_folder)
seedSetting = 'fgh'
trafficScaleSetting = 'asd'

## Df to csv

In [30]:
for index, row in result_df.iterrows():
    row_data = {
        'routeId': row['routeid'],
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['battery']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableBattery are excluded before concatenating the DataFrames
    tableBattery = tableBattery.dropna(axis=1, how='all')

    tableBattery = pd.concat([tableBattery, temp_df], ignore_index=True)


'id': group_id,
        'avgSpeed': avg_speed,
        'batteryCapacity': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'stops': count_stops

In [31]:
tableBattery.to_csv('batteryData.csv', index=False, sep=';')
#tableBattery

,routeId,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,330.0,nap_gellert_e,C622152254.0,dd,4.476667,107.76,1023.22,18.70,-10.67,wh,3,2821.226864
1,330.0,nap_gellert_e,C622155141.0,dd,4.791268,92.44,1023.22,18.62,-10.59,wh,3,2785.403274
2,9722.0,nap_gellert_e,C6305325.0,dd,4.905053,196.64,1866.71,17.58,-19.50,wh,4,1365.664043
3,2400.0,nap_gellert_e,C63598134.0,dd,4.487733,72.81,673.45,8.64,-10.76,wh,2,955.975080
4,2400.0,nap_gellert_e,C6359838.0,dd,4.182375,83.03,673.45,8.24,-10.86,wh,2,1062.622195
...,...,...,...,...,...,...,...,...,...,...,...,...
174,0085,nap_gellert_b,C78245617.0,fgh,3.679731,1007.18,8072.59,172.65,-218.99,asd,19,1725.077849
175,0085,nap_gellert_b,C78245945.0,fgh,4.521554,852.81,8690.53,240.24,-194.20,asd,18,2338.624497
176,1335,nap_gellert_b,C78257148.0,fgh,2.735513,751.02,3311.51,95.64,-98.63,asd,4,834.680594
177,1335,nap_gellert_b,C782571889.0,fgh,5.269959,122.40,2724.58,87.39,-83.98,asd,4,835.101168


In [ ]:
if __name__ == "__main__":
    main(seed = 0, scale = 0)